In [1]:
from geoip import geolite2
import pandas as pd
import csv
import googlemaps
import geopy
from collections import defaultdict

In [2]:
match = geolite2.lookup_mine()

In [3]:
match.location

(41.8471, -87.6248)

In [15]:
import time

def testkey(gmaps=None):
    try:
        ret = gmaps.geocode("Harbor Haus Crisis Services,cullman,al")
    except:
        ret = None
        
    return ret
    
keyfile = '/home/matt/googleapi.key'
def fix_the_key():
    goog_apikey = None
    while True:
        last_key = goog_apikey
        fh = open(keyfile,'r')
        goog_apikey = fh.readline().strip()
        fh.close()
        gmaps = geopy.GoogleV3(api_key=goog_apikey)
        #gmaps = googlemaps.Client(key=goog_apikey, read_timeout=3, retry_timeout=10)
        
        
        if testkey(gmaps):
            break
        else:    
            print "Bad key :("
            time.sleep(10)
            
    return gmaps

In [16]:
gmaps = fix_the_key()
gmaps.geocode("Harbor Haus Crisis Services,cullman,al")

Location((34.1748208, -86.8436124, 0.0))

In [6]:
#### for state in $STATES ; do
#    mkdir $state && rm -rf $state && mkdir $state
#
#    url="https://www.domesticshelters.org/${state}"
#    pushd $state
#    wget $url 2>/dev/null
#    city_links=`fgrep '<li><a href=' $state | grep program-shelters | sed -r 's/[^"]*"([^"]*)".*/\1/' | sed 's/^/https:\/\/www.domesticshelters.org/'`
#    for link in ${city_links} ; do
#        city=`echo $link | awk -F'/' '{print $5}'`
#        wget $link  -O - 2>/dev/null | grep 'h2><a hr' | awk -F'[/<>]' '{print $9","$6","$5}'
#    done
#    popd
#    exit
#done

In [17]:
centers_csv = open('/opt/projects/domestic_violence/all_centers.csv')

r = csv.DictReader(centers_csv)
cols = r.next()

In [18]:
if testkey(gmaps):
    print "All good!"
else:
    print "Something's fucked!"

all_centers = defaultdict(list)
for val in r.reader:
    name, city, state = val
    places_str = "%s, %s, %s" % (name, city, state)
    googmap = None

    for attempt in range(0,3):
        try:
            googmap = gmaps.geocode(places_str)
            all_centers[places_str] = { 'googmap': googmap,
                              'place': val }
            print "%s %s" % (len(all_centers), name)
            time.sleep(.2)

            break
        except Exception as e:
            print "Fixing? %s" % e
            fix_the_key()

All good!
1 2nd Chance.
2 The Salvation Army
3 The Lovelady Center
4 First Light
5 YWCA Domestic Violence Services
6 Legal Services Alabama
7 Harbor Haus Crisis Services
8 The House of Ruth.
9 Dothan Rescue Mission
10 Legal Services Alabama
11 Mary&#39;s Shelter
12 Safeplace.
13 Crisis Services of North Alabama
14 Downtown Rescue Mission
15 Legal Services Alabama
16 AshaKiran
17 Daybreak Family Resource Center of NW Alabama
18 Penelope House
19 Women In Crisis Shelter
20 Mission Of Hope
21 Legal Services Alabama
22 Family Sunshine Center
23 One Place Family Justice Center
24 Women of Refined Gold Inc
25 Friendship Mission
26 Legal Services Alabama
27 Domestic Violence Intervention Center
28 Legal Services Alabama
29 The Safehouse Of Shelby County
30 Crisis Center of Russell County
31 Lighthouse
32 SABRA Sanctuary.
33 Legal Services Alabama
34 Turning Point Domestic Violence &amp; Sexual Assault Services
35 Legal Services Alabama
36 Abused Women&#39;s Aid In Crisis. (AWAIC)
37 Clare Hou

In [118]:
df = pd.DataFrame([ {'lat': place['googmap'].latitude,
  'lng': place['googmap'].longitude,
  'addr': place['googmap'].address,
  'point': place['googmap'].point,
  'place': place['place'][0]} for place in all_centers.values() if type(place) is dict and place['googmap'] ])

In [155]:
def delta_lat(val):        
    return abs(val - match.location[0])

def delta_lng(val):        
    return abs(val - match.location[1])
#df
#lat_delta = df[['lat','lng']]
lat_deltas = df.lat.apply(delta_lat)
lng_deltas = df.lng.apply(delta_lng)

df['distance'] = abs(lat_deltas + lng_deltas)
#long_delta = df.lng.apply(delta, 'type'='lng')

In [158]:
df.sort(['distance']).values[:5]

/usr/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


array([[u'Chicago, IL, USA', 41.8781136, -87.6297982, 'KAN-WIN',
        Point(41.8781136, -87.6297982, 0.0), 0.03601180000000426],
       [u'Chicago, IL, USA', 41.8781136, -87.6297982,
        'Illinois Domestic Violence Hotline',
        Point(41.8781136, -87.6297982, 0.0), 0.03601180000000426],
       [u'Chicago, IL, USA', 41.8781136, -87.6297982,
        'Urban Family And Community Centers',
        Point(41.8781136, -87.6297982, 0.0), 0.03601180000000426],
       [u'Chicago, IL, USA', 41.8781136, -87.6297982,
        'Neopolitan Lighthouse', Point(41.8781136, -87.6297982, 0.0),
        0.03601180000000426],
       [u'Chicago, IL, USA', 41.8781136, -87.6297982,
        'Center for Advancing Domestic Peace.',
        Point(41.8781136, -87.6297982, 0.0), 0.03601180000000426]], dtype=object)

In [181]:
import folium
map_osm = folium.Map(location=[41.8781136, -87.6297982])
map_osm.create_map(path='osm.html')

for i in df.values:
    name = str(i[3]).decode('utf-8')
    lat,lng = (i[1], i[2])
    print name
    map_osm.simple_marker([float(lat), float(lng)], popup=name)
#map_1.simple_marker([45.3288, -121.6625], popup='Mt. Hood Meadows')

The Safe Place Inc
City Rescue Mission Of Saginaw
D.O.V.E.S. of Gateway.
Glynn Community Crisis Center
National Clearinghouse on Abuse in Later Life
Citizens Against Domestic Violence
MARYS House
Lower Valley Crisis Support Services
HAVIN
Crisis Center of Dodge City
Women Helping Battered Women
Life Crisis Center
Women&#39;s Shelter Of Central Arkansas
Centro de Ayuda a Víctimas de Violación
New Horizons
The Spouse Abuse Sexual Assault Crisis Center.
Domestic Abuse Shelter
Open Door Mission &amp; Lydia House
Area Agency on Aging Region One-Emergency Housing
Crospoint Human Services
Camden County Women&#39;s Center
Women In Distress Of Broward County
Branches Domestic Violence Shelter
Shepherd&#39;s House
Bonner County Homeless Task Force
Alaska Family Services
Alternatives to Violence of the Palouse
LACADA 
Domestic Violence Center of Gray&#39;s Harbor
Wayside Cross Gospel Rescue Mission
Bridges Community Services Transitional Housing
Opportunity House
Shalom Task Force
Women&#39;s Fre

/usr/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: Map.create_map is deprecated. Use Map.save instead
  app.launch_new_instance()
/usr/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: simple_marker is deprecated. Use add_children(Marker) instead


In [183]:
map_osm.create_map(path='osm.html')

/usr/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: Map.create_map is deprecated. Use Map.save instead
  if __name__ == '__main__':
